# Criando conjuntos de dados de descritores 2D e *fingerprints*

Tutorial simples de criação de conjuntos de dados contendo descritores 2D ou *fingerprints* moleculares. Adaptado de https://greglandrum.github.io/rdkit-blog/posts/2022-12-23-descriptor-tutorial.html

Começamos importando os módulos necessários

In [1]:
import rdkit
rdkit.__version__

'2022.03.3'

In [2]:
import numpy as np
import pandas as pd
from rdkit import Chem 
from rdkit.Chem import AllChem, Descriptors
from rdkit.Chem import rdFingerprintGenerator

Vamos utilizar o conjunto de dados [BACE do MoleculeNet](https://moleculenet.org/datasets-1), que contém dados de atividade quantitativa (pIC50) e qualitativa (rótulo binário: 0 para inativo ou 1 para ativo) para um conjunto de inibidores da β-secretase 1 humana (BACE-1).

Fiz o download do arquivo no formato .csv e coloquei na pasta `datasets`. Vamos importá-lo:

In [3]:
df = pd.read_csv("../datasets/bace.csv")
df.head()

,mol,CID,Class,Model,pIC50,MW,AlogP,HBA,HBD,RB,...,PEOE6 (PEOE6),PEOE7 (PEOE7),PEOE8 (PEOE8),PEOE9 (PEOE9),PEOE10 (PEOE10),PEOE11 (PEOE11),PEOE12 (PEOE12),PEOE13 (PEOE13),PEOE14 (PEOE14),canvasUID
0,O1CC[C@@H](NC(=O)[C@@H](Cc2cc3cc(ccc3nc2N)-c2c...,BACE_1,1,Train,9.154901,431.56979,4.4014,3,2,5,...,53.205711,78.640335,226.85541,107.43491,37.133846,0.000000,7.980170,0.0,0.000000,1
1,Fc1cc(cc(F)c1)C[C@H](NC(=O)[C@@H](N1CC[C@](NC(...,BACE_2,1,Train,8.853872,657.81073,2.6412,5,4,16,...,73.817162,47.171600,365.67694,174.07675,34.923889,7.980170,24.148668,0.0,24.663788,2
2,S1(=O)(=O)N(c2cc(cc3c2n(cc3CC)CC1)C(=O)N[C@H](...,BACE_3,1,Train,8.698970,591.74091,2.5499,4,3,11,...,70.365707,47.941147,192.40652,255.75255,23.654478,0.230159,15.879790,0.0,24.663788,3
3,S1(=O)(=O)C[C@@H](Cc2cc(O[C@H](COCC)C(F)(F)F)c...,BACE_4,1,Train,8.698970,591.67828,3.1680,4,3,12,...,56.657166,37.954151,194.35304,202.76335,36.498634,0.980913,8.188327,0.0,26.385181,4
4,S1(=O)(=O)N(c2cc(cc3c2n(cc3CC)CC1)C(=O)N[C@H](...,BACE_5,1,Train,8.698970,629.71283,3.5086,3,3,11,...,78.945702,39.361153,179.71288,220.46130,23.654478,0.230159,15.879790,0.0,26.100143,5


Várias colunas estão presentes. Somente nos interessam:
- "mol": a representação SMILES das estruturas
- "Model": indica uma separação entre conjunto de treinamento e teste feita pelo MoleculeNet (é um *scaffold split*, para mais detalhes, veja o Notebook [Data splits - selecionando um conjunto de teste](https://github.com/rflameiro/Python_e_Quiminformatica/blob/main/Quiminformatica/Data%20splits%20-%20selecionando%20um%20conjunto%20de%20teste.ipynb))
- "Class": classe binária indicando se o composto é ativo ou inativo no alvo (0 para inativo ou 1 para ativo)
- "pIC50": valor absoluto da atividade (logaritmo negativo do valor de IC50)

Agora, vamos usar as funções abaixo, feitas pelo autor do RDKit, para calcular os descritores.

In [4]:
# Funções úteis
# https://greglandrum.github.io/rdkit-blog/posts/2022-12-23-descriptor-tutorial.html


def getMolDescriptors(mol, missingVal=None):
    res = {}
    for nm,fn in Descriptors._descList:
        # algumas funções podem resultar em erros. Para esses casos, usa-se o try/except abaixo
        try:
            val = fn(mol)
        except:
            # Em caso de erro, uma mensagem será mostrada:
            import traceback
            traceback.print_exc()
            # e o valor do descritor será o de missingVal (no nosso exemplo, None)
            val = missingVal
        res[nm] = val
    return res


def calc_representation(smi_array, representation):
    # input: array contendo SMILES
    # representation: definir se quer descritores ("descs") ou fingerprints ("fps")
    # output: descritores/fingerprints no formato pd.DataFrame
    mols = [Chem.MolFromSmiles(smi) for smi in smi_array]
    if representation == "descs":
        descs = [getMolDescriptors(mol) for mol in mols]
        df_repr = pd.DataFrame(descs)
    elif representation == "fps":
        # Aqui você pode mudar o raio (radius) e o tamanho da fingerprint (fpSize)
        # Neste exemplo estamos usando somente os fingerprints Morgan. Outros estão disponíveis no RDKit
        mfpgen = rdFingerprintGenerator.GetMorganGenerator(radius=2, fpSize=1024)
        fps = [mfpgen.GetFingerprintAsNumPy(mol) for mol in mols]
        df_repr = pd.DataFrame(fps, columns=[str(i) for i in range(1024)])
    return df_repr

Agora vamos criar os conjuntos de dados. Nossos SMILES estão na coluna "mol":

In [5]:
smiles = df["mol"]

In [6]:
# Criar pd.Dataframe com descritores 2D
df_descs = calc_representation(smiles, representation="descs")

# Adicionar colunas relevantes
df_descs["SMILES"] = smiles
df_descs["Model"] = df["Model"]
df_descs["Class"] = df["Class"]
df_descs["pIC50"] = df["pIC50"]

# Exportar como .csv
df_descs.to_csv("../datasets/bace_descs2D.csv")

In [7]:
# Criar pd.Dataframe com fingerprints
df_fp = calc_representation(smiles, representation="fps")

# Adicionar colunas relevantes
df_fp["SMILES"] = smiles
df_fp["Model"] = df["Model"]
df_fp["Class"] = df["Class"]
df_fp["pIC50"] = df["pIC50"]

# Exportar como .csv
df_fp.to_csv("../datasets/bace_fingerprints.csv")

Agora temos arquivos .csv contendo os descritores 2D e os *fingerprints* para as estruturas do conjunto de dados BACE. Também adicionei colunas úteis contendo as estruturas químicas originais (SMILES), o *scaffold split* ("Model") e as atividades ("Class" e "pIC50").

Se quiser ver exemplos de como usar esses conjuntos para treinar modelos, veja os *Notebooks* da série "Calibração de modelos".